#  Churn Case Study

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Reading dataset
telecom_df = pd.read_csv('telecom_churn_data.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
telecom_df.head()

In [ ]:
# data check
telecom_df.info()

In [ ]:
telecom_df.shape

In [ ]:
telecom_df.describe()

In [ ]:
# Finding missing value %
100*telecom_df.isnull().sum()/len(telecom_df)

In [ ]:
# # We can impute the missing values of below columns with zero

telecom_df['total_rech_data_6'].fillna(0, inplace=True)
telecom_df['total_rech_data_7'].fillna(0, inplace=True)
telecom_df['total_rech_data_8'].fillna(0, inplace=True)
telecom_df['total_rech_data_9'].fillna(0, inplace=True)

telecom_df['av_rech_amt_data_6'].fillna(0, inplace=True)
telecom_df['av_rech_amt_data_7'].fillna(0, inplace=True)
telecom_df['av_rech_amt_data_8'].fillna(0, inplace=True)
telecom_df['av_rech_amt_data_9'].fillna(0, inplace=True)

In [ ]:
# Lets drop the columns which are having highest missing values and are not much useful

telecom_df.drop('last_date_of_month_6', axis=1, inplace=True)
telecom_df.drop('last_date_of_month_7', axis=1, inplace=True)
telecom_df.drop('last_date_of_month_8', axis=1, inplace=True)
t


In [ ]:
# Since in the problem statement it was mentioned that we are not considering revenue based churn, we can drop below columns

telecom_df.drop('arpu_2g_6', axis=1, inplace=True)
telecom_df.drop('arpu_2g_7', axis=1, inplace=True)
telecom_df.drop('arpu_2g_8', axis=1, inplace=True)
telecom_df.drop('arpu_2g_9', axis=1, inplace=True)

telecom_df.drop('arpu_3g_6', axis=1, inplace=True)
telecom_df.drop('arpu_3g_7', axis=1, inplace=True)
telecom_df.drop('arpu_3g_8', axis=1, inplace=True)
telecom_df.drop('arpu_3g_9', axis=1, inplace=True)

In [ ]:
# We can just take a copy of the original dataset
telecom_df_copy = telecom_df.copy()

In [ ]:
# We can impute the missing values with either mean / median / Iterative Imputer. Let's try with II

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

telecom_columns = telecom_df.columns

In [ ]:
ii = IterativeImputer()
telecom_df_clean = pd.DataFrame(ii.fit_transform(telecom_df_copy))
telecom_df_clean.columns = telecom_columns
telecom_df_clean.head()

In [ ]:
telecom_df.shape

In [ ]:
telecom_df_clean.isnull().sum()

In [ ]:
# Lets check if the iterative imputer imputes any negative values in any non-negative columns

plt.figure(figsize=[20,5])
plt.subplot(1,2,1)
sns.boxplot(y=telecom_df.onnet_mou_6)
plt.subplot(1,2,2)
sns.boxplot(y=telecom_df_clean.onnet_mou_6)

##### If we closely observe, there are some negative values in minutes of usage which doesn't make any sense. So, let's delete those rows

In [ ]:
# So, check for negative values created by iterative imputer and replace them with 0

telecom_df_clean[telecom_df_clean < 0] = 0

In [ ]:
telecom_df_clean.shape

## Filter HVC

In [ ]:
# Highest Value Customers are those who are producing revenue who have recharged with an amount more than or equal to X, 
# where X is the 70th percentile of the average recharge amount in the first two months (the good phase).

# To do that, we need to derive total amount of data recharge from the avg data amt recharge and total data recharges

# Calculate call recharge + data recharge
telecom_df_clean['total_amount_6'] = telecom_df_clean['total_rech_amt_6'] + (telecom_df_clean['av_rech_amt_data_6']*telecom_df_clean['total_rech_data_6'])
telecom_df_clean['total_amount_7'] = telecom_df_clean['total_rech_amt_7'] + (telecom_df_clean['av_rech_amt_data_7']*telecom_df_clean['total_rech_data_7'])

# Calculate the average recharge of June and July and also calculate the 70% of the value
telecom_df_clean['average_rech_6_7'] = (telecom_df_clean['total_amount_6'] + telecom_df_clean['total_amount_7'])/2
cut_off_hvc = telecom_df_clean['average_rech_6_7'].quantile(0.7)
print(cut_off_hvc)

In [ ]:
# Now, filter out the High Value Customers from all customers and then will predict the churn for them

telecom_df_hvc = telecom_df_clean[telecom_df_clean['average_rech_6_7'] >= cut_off_hvc]

In [ ]:
telecom_df_hvc.shape

## Deriving Churn column

In [ ]:
telecom_df_hvc['total_ic_og_roam_mou_9'] = telecom_df_hvc['total_og_mou_9'] + telecom_df_hvc['total_ic_mou_9']
telecom_df_hvc['total_data_vol_mb_9'] = telecom_df_hvc['vol_2g_mb_9'] + telecom_df_hvc['vol_3g_mb_9']

telecom_df_hvc['Churn'] = np.where((telecom_df_hvc['total_ic_og_roam_mou_9'] + telecom_df_hvc['total_data_vol_mb_9']) <= 0 , 1 , 0)
telecom_df_hvc.head()

In [ ]:
telecom_df_hvc['Churn'].value_counts()

In [ ]:
# Calculate churn percentage

print("Churn percentage is: ",(telecom_df_hvc['Churn'].sum() / len(telecom_df_hvc))*100)

## Dropping 9th month data

In [ ]:
# Take a copy and delete all 9th month data

telecom_df_hvc_copy = telecom_df_hvc.copy()
telecom_df_hvc = telecom_df_hvc.loc[:, ~telecom_df_hvc.columns.str.endswith('_9')]
telecom_df_hvc.head()

In [ ]:
telecom_df_hvc.shape

## Data Preparation

In [ ]:
# Check for outliers in some continous variable columns

telecom_df_hvc.describe(percentiles=[.25,.5,.75,.90,.95,.99])

In [ ]:
# From the above, we can clearly see there are outliers in almost all columns. Lets plot some and confirm

plt.figure(figsize=[20,15])
plt.subplot(3,3,1)
sns.boxplot(y=telecom_df_hvc.arpu_6)
plt.subplot(3,3,2)
sns.boxplot(y=telecom_df_hvc.onnet_mou_6)
plt.subplot(3,3,3)
sns.boxplot(y=telecom_df_hvc.onnet_mou_6)
plt.subplot(3,3,4)
sns.boxplot(y=telecom_df_hvc.offnet_mou_6)
plt.subplot(3,3,5)
sns.boxplot(y=telecom_df_hvc.roam_ic_mou_6)
plt.subplot(3,3,6)
sns.boxplot(y=telecom_df_hvc.roam_og_mou_6)
plt.subplot(3,3,7)
sns.boxplot(y=telecom_df_hvc.total_og_mou_6)
plt.subplot(3,3,8)
sns.boxplot(y=telecom_df_hvc.total_ic_mou_6)
plt.subplot(3,3,9)
sns.boxplot(y=telecom_df_hvc.average_rech_6_7)

In [ ]:
# We can either delete / treat / cap outliers based on our need. 

# Since, we have less data, we can now cap the outliers for all columns at once with below piece of code

cols = telecom_df_hvc.columns
# replacing the outliers with 99% value
for col in cols[1:]:
    Q3 = telecom_df_hvc[col].quantile(.99)
    telecom_df_hvc[col] = telecom_df_hvc[col].replace(telecom_df_hvc[col][(telecom_df_hvc[col]>Q3)], Q3)

In [ ]:
# After capping outliers

telecom_df_hvc.describe(percentiles=[.25,.5,.75,.90,.95,.99])

In [ ]:
# Let's check the correlation coefficients to see which variables are highly correlated
# For better visualization of heatmap we are dividing the dataframe into two keeping the dependent variable in common

telecom_df_hvc_1 = telecom_df_hvc.iloc[:, :30]
telecom_df_hvc_2 = telecom_df_hvc.iloc[:, 31:]
telecom_df_hvc_1['Churn'] = telecom_df_hvc_2['Churn']

In [ ]:
plt.figure(figsize = (25, 15))
sns.heatmap(telecom_df_hvc_1.corr(), annot = True, cmap="YlGnBu")
plt.show()

##### From the above plot, we could see that total recharge amount and average revenue per user are highly correlated

In [ ]:
plt.figure(figsize = (25, 15))
sns.heatmap(telecom_df_hvc_2.corr(), annot = True, cmap="YlGnBu")
plt.show()

###### From the above plot, Sachet and total recharge are highly correlated

In [ ]:
# Train - Test Split

from sklearn.model_selection import train_test_split

In [ ]:
# Take a copy before splitting
telecom_df_hvc_split = telecom_df_hvc.copy()

In [ ]:
# Putting feature variable to X

X = telecom_df_hvc.drop(['mobile_number','Churn'],axis=1)
X.head()

In [ ]:
# Putting response variable to y

y = telecom_df_hvc['Churn']

y.head()

In [ ]:
# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=100)

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE()
X_smote,y_smote = sm.fit_sample(X_train,y_train)

In [ ]:
print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)

In [ ]:
X_train = pd.DataFrame(X_smote, columns=X_train.columns)

In [ ]:
X_train.head(10)

In [ ]:
y_train = pd.DataFrame(y_smote, columns = ['Churn'])


In [ ]:
y_train.head(10)


In [ ]:
# Let's apply scaling on training data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

for col in X.columns:
    X_train[[col]] = scaler.fit_transform(X_train[[col]])

In [ ]:
X_train.head()

### Feature Selection using RFE

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [ ]:
# Running RFE with the output number of the variable equal to 15

from sklearn.feature_selection import RFE
rfe = RFE(logreg, 15)             # running RFE with 15 variables as output
rfe = rfe.fit(X_train, y_train)

In [ ]:
rfe.support_

In [ ]:
list(zip(X_train.columns, rfe.support_, rfe.ranking_))

In [ ]:
colm = X_train.columns[rfe.support_]

In [ ]:
X_train.columns[~rfe.support_]

###### Asseessing the model with statsmodel

In [ ]:
import statsmodels.api as sm

X_train_sm = sm.add_constant(X_train[colm])
logm1 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm1.fit()
res.summary()

In [ ]:
# Checking VIF's

# Check for the VIF values of the feature variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train[colm].columns
vif['VIF'] = [variance_inflation_factor(X_train[colm].values, i) for i in range(X_train[colm].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

There are a few variables with high VIF. It's best to drop these variables as they aren't helping much with prediction and unnecessarily making the model complex. The variable 'total_og_mou_8' has the highest VIF. So let's start by dropping that.

In [ ]:
colm = colm.drop('total_og_mou_8', 1)
colm

In [ ]:
# Let's re-run the model using the selected variables
X_train_sm = sm.add_constant(X_train[colm])
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train[colm].columns
vif['VIF'] = [variance_inflation_factor(X_train[colm].values, i) for i in range(X_train[colm].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# Getting the predicted values on the train set
y_train_pred = res.predict(X_train_sm)
y_train_pred = y_train_pred.values.reshape(-1)
y_train_pred[:10]

In [ ]:
# y_train_pred_final = pd.DataFrame({'Churn':y_train.values, 'Churn_Prob':y_train_pred})
y_train_pred_final = pd.DataFrame(y_train_pred, columns = ['Churn_Prob'])
# y_train_pred_final = y_train_pred_final.rename(columns = {'0': 'Churn Prob'}, inplace = True)

In [ ]:
y_train_pred_final['mbl_num'] = y_train.index


In [ ]:
y_train_pred_final = pd.concat([y_train_pred_final, y_train], join = 'outer', axis = 1)

In [ ]:
y_train_pred_final['predicted'] = y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.5 else 0)

# Let's see the head
y_train_pred_final.head()

In [ ]:
# Let's take a look at the confusion matrix again 
from sklearn import metrics
confusion = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.predicted )
confusion

In [ ]:
# Actual/Predicted     not_churn    churn
        # not_churn        19138     171
        # churn            1314      377  

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted)

In [ ]:
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
# Calculate false postive rate - predicting churn when customer does not have churned
print(FP/ float(TN+FP))

In [ ]:
# positive predictive value 
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

### Plotting the ROC Curve

In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve( y_train_pred_final.Churn, y_train_pred_final.Churn_Prob, drop_intermediate = False )

In [ ]:
draw_roc(y_train_pred_final.Churn, y_train_pred_final.Churn_Prob)

#### Finding Optimal Cutoff Point

In [ ]:
# Optimal cutoff probability is that prob where we get balanced sensitivity and specificity

# Let's create columns with different probability cutoffs 
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_train_pred_final[i]= y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > i else 0)
y_train_pred_final.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
from sklearn.metrics import confusion_matrix

# TP = confusion[1,1] # true positive 
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
plt.show()

#### From the curve above, 0.1 is the optimum point to take it as a cutoff probability.

In [ ]:
y_train_pred_final['final_predicted'] = y_train_pred_final.Churn_Prob.map( lambda x: 1 if x > 0.55 else 0)

y_train_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.final_predicted)

In [ ]:
confusion2 = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.final_predicted )
confusion2

In [ ]:
TP = confusion2[1,1] # true positive 
TN = confusion2[0,0] # true negatives
FP = confusion2[0,1] # false positives
FN = confusion2[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
# Calculate false postive rate - predicting churn when customer does not have churned
print(FP/ float(TN+FP))

In [ ]:
# Positive predictive value 
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

#### Precision and Recall

###### Precision
###### TP / TP + FP

In [ ]:
confusion[1,1]/(confusion[0,1]+confusion[1,1])

###### Recall
###### TP / TP + FN

In [ ]:
confusion[1,1]/(confusion[1,0]+confusion[1,1])

In [ ]:
from sklearn.metrics import precision_score, recall_score

In [ ]:
#?precision_score

In [ ]:
precision_score(y_train_pred_final.Churn, y_train_pred_final.predicted)

In [ ]:
recall_score(y_train_pred_final.Churn, y_train_pred_final.predicted)

##### Precision and recall tradeoff

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
y_train_pred_final.Churn, y_train_pred_final.predicted

In [ ]:
p, r, thresholds = precision_recall_curve(y_train_pred_final.Churn, y_train_pred_final.Churn_Prob)

In [ ]:
plt.plot(thresholds, p[:-1], "g-")
plt.plot(thresholds, r[:-1], "r-")
plt.show()

In [ ]:
### Making predictions on the test set

for c in colm:
    X_test[[c]] = scaler.transform(X_test[[c]])
X_test = X_test[colm]
X_test_sm = sm.add_constant(X_test)

In [ ]:
y_test_pred = res.predict(X_test_sm)

In [ ]:
y_test_pred[:10]

In [ ]:
# Converting y_pred to a dataframe which is an array
y_pred_1 = pd.DataFrame(y_test_pred)

In [ ]:
# Let's see the head
y_pred_1.head()

In [ ]:
# Converting y_test to dataframe
y_test_df = pd.DataFrame(y_test)

In [ ]:
# Putting CustID to index
y_test_df['mbl_num'] = y_test_df.index

In [ ]:
# Removing index for both dataframes to append them side by side 
y_pred_1.reset_index(drop=True, inplace=True)
y_test_df.reset_index(drop=True, inplace=True)

In [ ]:
# Appending y_test_df and y_pred_1
y_pred_final = pd.concat([y_test_df, y_pred_1],axis=1)

In [ ]:
y_pred_final.head()

In [ ]:
# Renaming the column 
y_pred_final= y_pred_final.rename(columns={ 0 : 'Churn_Prob'})

In [ ]:
# Rearranging the columns
y_pred_final = y_pred_final.reindex(['mbl_num','Churn','Churn_Prob'], axis=1)

In [ ]:
y_pred_final.head()

In [ ]:
y_pred_final['final_predicted'] = y_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.58 else 0)

In [ ]:
y_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_pred_final.Churn, y_pred_final.final_predicted)

In [ ]:
confusion2 = metrics.confusion_matrix(y_pred_final.Churn, y_pred_final.final_predicted )
confusion2

In [ ]:
TP = confusion2[1,1] # true positive 
TN = confusion2[0,0] # true negatives
FP = confusion2[0,1] # false positives
FN = confusion2[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

## Logistic Regression using PCA

In [ ]:
# Putting feature variable to X

X = telecom_df_hvc.drop(['mobile_number','Churn'],axis=1)
X.head()

In [ ]:
# Putting response variable to y
y = telecom_df_hvc['Churn']
y.head()

In [ ]:
# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7,test_size=0.3,random_state=100)

In [ ]:
# getting rid of imbalancing using SMOTE

from imblearn.over_sampling import SMOTE

sm = SMOTE()
X_train,y_train = sm.fit_sample(X_train,y_train)

In [ ]:
print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)

In [ ]:
# Import PCA module
from sklearn.decomposition import PCA

In [ ]:
# Create an instance of PCA
pca = PCA(random_state=42)

In [ ]:
pca.fit(X_train)

In [ ]:
pca.components_

In [ ]:
pca.explained_variance_ratio_

In [ ]:
# Bar-plot explaining variance
plt.figure(figsize=(12,6))
plt.title('Bar-plot explaining variance')
plt.xlabel('Number of componenets')
plt.bar(range(1,len(pca.explained_variance_ratio_)+1), pca.explained_variance_ratio_)
plt.show()

In [ ]:
var_cumu = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
# Scree plot
fig = plt.figure(figsize=[12,8])
plt.vlines(x=15, ymax=1, ymin=0, colors="r", linestyles="--")
plt.hlines(y=0.95, xmax=30, xmin=0, colors="g", linestyles="--")
plt.plot(var_cumu)
plt.ylabel("Cumulative variance explained")
plt.show()

From the above bar plot and Scree plot, we can say that the explained variance raitio is around 95% at 16 components 

In [ ]:
# Let's take n_components =16
from sklearn.decomposition import IncrementalPCA
pca_final = IncrementalPCA(n_components = 16)

In [ ]:
newdata = pca_final.fit_transform(X_train)

In [ ]:
newdata.shape

Making a dataframe out of it for convenience

In [ ]:
columns=[]
for i in range(1,newdata.shape[1]+1):
  columns.append("PC"+str(i))
pca_df = pd.DataFrame(newdata, columns=columns)
pca_df.shape

In [ ]:
pca_df.head()

In [ ]:
corrmat = np.corrcoef(newdata.transpose())
corrmat.shape

In [ ]:
plt.figure(figsize=[15,15])
sns.heatmap(corrmat, annot=True)

In [ ]:
X_test_pca = pca_final.transform(X_test)
X_test_pca.shape

### Applying Logistic Regression on the data on our prinicipal components

In [ ]:
from sklearn.linear_model import LogisticRegression
learner_pca = LogisticRegression()
model_pca = learner_pca.fit(newdata, y_train)

Making predictions on the test set

In [ ]:
pred_probs_test = model_pca.predict_proba(X_test_pca)
"{:2.2}".format(metrics.roc_auc_score(y_test, pred_probs_test[:,1]))

In [ ]:
pca_again = PCA(0.9)
newdata2 = pca_again.fit_transform(X_train)
newdata2.shape

In [ ]:
learner_pca2 = LogisticRegression()
model_pca2 = learner_pca2.fit(newdata2, y_train)
X_test_pca2 = pca_again.transform(X_test)
X_test_pca2.shape

In [ ]:
pred_probs_test2 = model_pca2.predict_proba(X_test_pca2)[:,1]
"{:2.2}".format(metrics.roc_auc_score(y_test, pred_probs_test2))

# Decision Tree using PCA

In [ ]:
# Importing decision tree classifier from sklearn library
from sklearn.tree import DecisionTreeClassifier

# Fitting the decision tree with default hyperparameters, apart from
# max_depth which is 5 so that we can plot and read the tree.
dt_default = DecisionTreeClassifier(max_depth=5)
dt_default.fit(newdata, y_train)

In [ ]:
# Let's check the evaluation metrics of our default model

# Importing classification report and confusion matrix from sklearn metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Making predictions
y_pred_default = dt_default.predict(X_test_pca)

# Printing classification report
print(classification_report(y_test, y_pred_default))

In [ ]:
# Printing confusion matrix and accuracy
confusion=confusion_matrix(y_test,y_pred_default)
print(confusion)
print(accuracy_score(y_test,y_pred_default))

In [ ]:
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

### Plotting the Decision Tree

We need the ```graphviz``` library to plot a tree.

In [ ]:

# Importing required packages for visualization
from IPython.display import Image  
from six import StringIO  
from sklearn.tree import export_graphviz
import pydotplus, graphviz

# Putting features
features = list(pca_df.columns)
features

In [ ]:
# plotting tree with max_depth=3
dot_data = StringIO()  
export_graphviz(dt_default, out_file=dot_data,
                feature_names=features, filled=True,rounded=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
### Hyperparameter Tuning

### Tuning max_depth

# GridSearchCV to find optimal max_depth
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5

# parameters to build the model on
parameters = {'max_depth': range(1, 40)}

# instantiate the model
dtree = DecisionTreeClassifier(criterion = "gini", 
                               random_state = 100)

# fit tree on training data
tree = GridSearchCV(dtree, parameters, 
                    cv=n_folds,
                    return_train_score='warn', 
                   scoring="accuracy")
tree.fit(newdata, y_train)

In [ ]:
# scores of GridSearch CV
scores = tree.cv_results_
pd.DataFrame(scores).head()

Now let's visualize how train and test score changes with max_depth.

In [ ]:
# plotting accuracies with max_depth
plt.figure()
plt.plot(scores["param_max_depth"], 
         scores["mean_train_score"], 
         label="training accuracy")
plt.plot(scores["param_max_depth"], 
         scores["mean_test_score"], 
         label="test accuracy")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

You can see that as we increase the value of max_depth, both training and test score increase till about max-depth = 4, after which the test score gradually reduces. Note that the scores are average accuracies across the 5-folds. 

Thus, it is clear that the model is overfitting the training data if the max_depth is too high. Next, let's see how the model behaves with other hyperparameters.

In [ ]:
### Tuning min_samples_leaf

# GridSearchCV to find optimal max_depth
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5

# parameters to build the model on
parameters = {'min_samples_leaf': range(5, 200, 20)}

# instantiate the model
dtree = DecisionTreeClassifier(criterion = "gini", 
                               random_state = 100)

# fit tree on training data
tree = GridSearchCV(dtree, parameters, 
                    cv=n_folds,
                    return_train_score='warn', 
                   scoring="accuracy")
tree.fit(newdata, y_train)

In [ ]:
# scores of GridSearch CV
scores = tree.cv_results_
pd.DataFrame(scores).head()

In [ ]:
# plotting accuracies with min_samples_leaf
plt.figure()
plt.plot(scores["param_min_samples_leaf"], 
         scores["mean_train_score"], 
         label="training accuracy")
plt.plot(scores["param_min_samples_leaf"], 
         scores["mean_test_score"], 
         label="test accuracy")
plt.xlabel("min_samples_leaf")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

You can see that at low values of min_samples_leaf, the tree gets a bit overfitted. At values > 100, however, the model becomes more stable and the training and test accuracy start to converge

In [ ]:
### Tuning min_samples_split

# GridSearchCV to find optimal min_samples_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5

# parameters to build the model on
parameters = {'min_samples_split': range(5, 200, 20)}

# instantiate the model
dtree = DecisionTreeClassifier(criterion = "gini", 
                               random_state = 100)

# fit tree on training data
tree = GridSearchCV(dtree, parameters, 
                    cv=n_folds,
                    return_train_score='warn', 
                   scoring="accuracy")
tree.fit(newdata, y_train)

In [ ]:
# scores of GridSearch CV
scores = tree.cv_results_
pd.DataFrame(scores).head()

In [ ]:
# plotting accuracies with min_samples_leaf
plt.figure()
plt.plot(scores["param_min_samples_split"], 
         scores["mean_train_score"], 
         label="training accuracy")
plt.plot(scores["param_min_samples_split"], 
         scores["mean_test_score"], 
         label="test accuracy")
plt.xlabel("min_samples_split")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

This shows that as you increase the min_samples_split, the tree overfits lesser since the model is less complex.

## Grid Search to Find Optimal Hyperparameters

We can now use GridSearchCV to find multiple optimal hyperparameters together. Note that this time, we'll also specify the criterion (gini/entropy or IG).

In [ ]:
# Create the parameter grid 
param_grid = {
    'max_depth': range(5, 15, 5),
    'min_samples_leaf': range(50, 150, 50),
    'min_samples_split': range(50, 150, 50),
    'criterion': ["entropy", "gini"]
}

n_folds = 5

# Instantiate the grid search model
dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid = param_grid, 
                          cv = n_folds, verbose = 1)

# Fit the grid search to the data
grid_search.fit(newdata,y_train)

In [ ]:
# cv results
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results

In [ ]:
# printing the optimal accuracy score and hyperparameters
print("best accuracy", grid_search.best_score_)
print(grid_search.best_estimator_)

**Running the model with best parameters obtained from grid search.**

In [ ]:
# model with optimal hyperparameters
clf_gini = DecisionTreeClassifier(criterion = "gini", 
                                  random_state = 100,
                                  max_depth=10, 
                                  min_samples_leaf=50,
                                  min_samples_split=100)
clf_gini.fit(newdata, y_train)

In [ ]:
# accuracy score
clf_gini.score(X_test_pca,y_test)

In [ ]:

dot_data = StringIO()  
export_graphviz(clf_gini, out_file=dot_data,feature_names=features,filled=True,rounded=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

You can see that this tree is too complex to understand. Let's try reducing the max_depth and see how the tree looks.

In [ ]:
# tree with max_depth = 3
clf_gini = DecisionTreeClassifier(criterion = "gini", 
                                  random_state = 100,
                                  max_depth=3, 
                                  min_samples_leaf=50,
                                  min_samples_split=100)
clf_gini.fit(newdata, y_train)

# score
print(clf_gini.score(X_test_pca,y_test))

In [ ]:
# plotting tree with max_depth=3

dot_data = StringIO()  
export_graphviz(clf_gini, out_file=dot_data,feature_names=features,filled=True,rounded=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
# classification metrics
from sklearn.metrics import classification_report,confusion_matrix
y_pred = clf_gini.predict(X_test_pca)
print(classification_report(y_test, y_pred))

In [ ]:
# confusion matrix
confusion=confusion_matrix(y_test,y_pred)
print(confusion)

In [ ]:
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

# Random Forest using PCA



#### Default Hyperparameters
Let's first fit a random forest model with default hyperparameters.

In [ ]:
# Importing random forest classifier from sklearn library
from sklearn.ensemble import RandomForestClassifier

# Running the random forest with default parameters.
rfc = RandomForestClassifier()

In [ ]:
# fit
rfc.fit(newdata,y_train)

In [ ]:
# Making predictions
predictions = rfc.predict(X_test_pca)

In [ ]:
# Importing classification report and confusion matrix from sklearn metrics
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

In [ ]:
# Let's check the report of our default model
print(classification_report(y_test,predictions))

In [ ]:
# Printing confusion matrix
confusion=confusion_matrix(y_test,predictions)
print(confusion)

In [ ]:
print(accuracy_score(y_test,predictions))

In [ ]:
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

### Hyperparameter Tuning

In [ ]:
### Tuning max_depth

# GridSearchCV to find optimal n_estimators
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5

# parameters to build the model on
parameters = {'max_depth': range(2, 20, 5)}

# instantiate the model
rf = RandomForestClassifier()


# fit tree on training data
rf = GridSearchCV(rf, parameters, 
                    cv=n_folds,
                    return_train_score='warn', 
                   scoring="accuracy")
rf.fit(newdata, y_train)

In [ ]:
# scores of GridSearch CV
scores = rf.cv_results_
pd.DataFrame(scores).head()

In [ ]:
# plotting accuracies with max_depth
plt.figure()
plt.plot(scores["param_max_depth"], 
         scores["mean_train_score"], 
         label="training accuracy")
plt.plot(scores["param_max_depth"], 
         scores["mean_test_score"], 
         label="test accuracy")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

You can see that as we increase the value of max_depth, both train and test scores increase till a point, but after that test score starts to decrease. The ensemble tries to overfit as we increase the max_depth.

Thus, controlling the depth of the constituent trees will help reduce overfitting in the forest.

In [ ]:
### Tuning n_estimators

# GridSearchCV to find optimal n_estimators
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5

# parameters to build the model on
parameters = {'n_estimators': range(50, 500, 200)}

# instantiate the model (note we are specifying a max_depth)
rf = RandomForestClassifier(max_depth=4)


# fit tree on training data
rf = GridSearchCV(rf, parameters, 
                    cv=n_folds, 
                    return_train_score='warn',
                   scoring="accuracy")
rf.fit(newdata, y_train)

In [ ]:
# scores of GridSearch CV
scores = rf.cv_results_
pd.DataFrame(scores).head()

In [ ]:
# plotting accuracies with n_estimators
plt.figure()
plt.plot(scores["param_n_estimators"], 
         scores["mean_train_score"], 
         label="training accuracy")
plt.plot(scores["param_n_estimators"], 
         scores["mean_test_score"], 
         label="test accuracy")
plt.xlabel("n_estimators")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
### Tuning max_features

# Let's see how the model performance varies with ```max_features```, which is the maximum numbre of features considered for splitting at a node.

# GridSearchCV to find optimal max_features
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5

# parameters to build the model on
parameters = {'max_features': [0.2, 0.5, 0.75]}

# instantiate the model
rf = RandomForestClassifier(max_depth=4)


# fit tree on training data
rf = GridSearchCV(rf, parameters, 
                    cv=n_folds, 
                   return_train_score='warn',
                   scoring="accuracy")
rf.fit(newdata, y_train)

In [ ]:
# scores of GridSearch CV
scores = rf.cv_results_
pd.DataFrame(scores).head()

In [ ]:
# plotting accuracies with max_features
plt.figure()
plt.plot(scores["param_max_features"], 
         scores["mean_train_score"], 
         label="training accuracy")
plt.plot(scores["param_max_features"], 
         scores["mean_test_score"], 
         label="test accuracy")
plt.xlabel("max_features")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

Apparently, the training and test scores *both* seem to increase as we increase max_features, and the model doesn't seem to overfit more with increasing max_features. Think about why that might be the case.

In [ ]:
### Tuning min_samples_leaf

# GridSearchCV to find optimal min_samples_leaf
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5

# parameters to build the model on
parameters = {'min_samples_leaf': range(100, 400, 50)}

# instantiate the model
rf = RandomForestClassifier()


# fit tree on training data
rf = GridSearchCV(rf, parameters, 
                    cv=n_folds, 
                  return_train_score='warn',
                   scoring="accuracy")
rf.fit(newdata, y_train)

In [ ]:
# scores of GridSearch CV
scores = rf.cv_results_
pd.DataFrame(scores).head()

In [ ]:
# plotting accuracies with min_samples_leaf
plt.figure()
plt.plot(scores["param_min_samples_leaf"], 
         scores["mean_train_score"], 
         label="training accuracy")
plt.plot(scores["param_min_samples_leaf"], 
         scores["mean_test_score"], 
         label="test accuracy")
plt.xlabel("min_samples_leaf")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

You can see that the model starts of overfit as you decrease the value of min_samples_leaf. 

In [ ]:
### Tuning min_samples_split

# Let's now look at the performance of the ensemble as we vary min_samples_split.

# GridSearchCV to find optimal min_samples_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5

# parameters to build the model on
parameters = {'min_samples_split': range(200, 500, 50)}

# instantiate the model
rf = RandomForestClassifier()


# fit tree on training data
rf = GridSearchCV(rf, parameters, 
                    cv=n_folds,
                    return_train_score='warn',
                   scoring="accuracy")
rf.fit(newdata, y_train)

In [ ]:
# scores of GridSearch CV
scores = rf.cv_results_
pd.DataFrame(scores).head()

In [ ]:
# plotting accuracies with min_samples_split
plt.figure()
plt.plot(scores["param_min_samples_split"], 
         scores["mean_train_score"], 
         label="training accuracy")
plt.plot(scores["param_min_samples_split"], 
         scores["mean_test_score"], 
         label="test accuracy")
plt.xlabel("min_samples_split")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

## Grid Search to Find Optimal Hyperparameters

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [4,8,10],
    'min_samples_leaf': range(100, 400, 200),
    'min_samples_split': range(200, 500, 200),
    'n_estimators': [100,200, 300], 
    'max_features': [5, 10]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1,verbose = 1)

In [ ]:
# Fit the grid search to the data
grid_search.fit(newdata, y_train)

In [ ]:
# printing the optimal accuracy score and hyperparameters
print('We can get accuracy of',grid_search.best_score_,'using',grid_search.best_params_)

# Observations and recommendations

#### After using the RFE in Logistic regression without using PCA the top variables are

total_ic_mou_8
total_rech_num_8
total_ic_mou_7
total_rech_num_7
arpu_7
offnet_mou_8
sachet_2g_8
last_day_rch_amt_8
onnet_mou_8
vol_2g_mb_8
roam_og_mou_8
monthly_2g_8
monthly_3g_8
sep_vbc_3g

##### After using the Logistic regression on PCA variables we got the roc_auc_score = 0.86.

##### After building the model using Decision tree with best parameters obtained from grid search we got
    Accuracy = 0.802 with depth = 10
    Accuracy = 0.72, specificity = 0.73 with depth = 3
After building the model using Random forest on PCA variables we got the accuracy as 0.83
